In [1]:
# 05_build_master_dataset.ipynb
# Цель ноутбука:
# 1. Собрать все player_standard_stats.csv по всем сезонам и лигам
# 2. Привести к единому формату
# 3. Сохранить единый master players dataset -> data/processed/players/players_all.csv

import pandas as pd
import numpy as np
from pathlib import Path
import re
from typing import List

# базовая директория проекта (ноутбук лежит в /notebooks)
BASE_DIR = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

RAW_DIR = BASE_DIR / "data" / "raw" / "fbref"
OUT_PLAYERS = BASE_DIR / "data" / "processed" / "players" / "players_all.csv"

# вот эти лиги считаем "основными" на сейчас
LEAGUES = ["epl", "laliga", "bundesliga"]

print("BASE_DIR =", BASE_DIR)
print("RAW_DIR  =", RAW_DIR)
print("OUT_PLAYERS =", OUT_PLAYERS)
print("Лиги для сборки:", LEAGUES)

BASE_DIR = /Users/kekc/Projects/GIT/sports-stats-analysis
RAW_DIR  = /Users/kekc/Projects/GIT/sports-stats-analysis/data/raw/fbref
OUT_PLAYERS = /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/players/players_all.csv
Лиги для сборки: ['epl', 'laliga', 'bundesliga']


In [2]:
# Вспомогательные функции для загрузки и очистки

def _extract_season_from_path(p: Path) -> str:
    """
    Получаем строку сезона вида '2024-2025' из пути вроде:
    data/raw/fbref/epl_2024-2025/player_standard_stats.csv
    """
    # берем имя папки: epl_2024-2025
    folder_name = p.parent.name
    # режем по "_" и берём вторую часть
    # на всякий случай fallback = "" если вдруг формат другой
    return folder_name.split("_", 1)[1] if "_" in folder_name else ""


def load_player_stats_for_league(league_code: str) -> pd.DataFrame:
    """
    Читает все player_standard_stats.csv для заданной лиги (epl / laliga / bundesliga),
    добавляет league и season.
    Возвращает один склеенный DataFrame.
    """
    rows = []

    # пример пути: RAW_DIR / "epl_2024-2025" / "player_standard_stats.csv"
    for folder in RAW_DIR.glob(f"{league_code}_*"):
        csv_path = folder / "player_standard_stats.csv"
        if not csv_path.exists():
            # вдруг для сезона не собрали игроков => пропускаем
            continue

        df_season = pd.read_csv(csv_path)
        season = _extract_season_from_path(csv_path)

        df_season["season"] = season
        df_season["league"] = league_code
        rows.append(df_season)

        print(f"[LOAD] {csv_path.relative_to(BASE_DIR)}  shape={df_season.shape}")

    if not rows:
        print(f"[WARN] Лига {league_code}: файлов не найдено")
        return pd.DataFrame()

    return pd.concat(rows, ignore_index=True)


def _to_number(val):
    """
    Преобразует строковые значения вида '12', '1,234', '0.45', '' в число (float),
    иначе возвращает NaN.
    """
    if pd.isna(val):
        return np.nan
    if isinstance(val, (int, float)):
        return val
    s = str(val).strip()
    if s == "" or s == "-":
        return np.nan
    # убираем запятые из тысячных
    s = s.replace(",", "")
    # уберём плюсы и прочие символы в скобках, если попадутся
    s = re.sub(r"[^\d\.\-]", "", s)
    try:
        return float(s)
    except ValueError:
        return np.nan


def clean_players_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Приводит объединённый датафрейм игроков к чистому виду:
    - нормализуем возраст (age может содержать '22-123' → берём левую часть)
    - числовые столбцы переводим в float
    - убираем заведомо агрегированные строки типа 'Squad Total' (идентифицируем по mp/min==NaN или player=="Squad Total")
    - дропаем дубликаты по (season, player, squad)
    """

    out = df.copy()

    # --- возраст ---
    # age может быть "22-123" (возраст-дни), берём всё до дефиса
    if "age" in out.columns:
        out["age"] = (
            out["age"]
            .astype(str)
            .str.extract(r"^(\d+)", expand=False)
            .astype(float)
        )

    # --- числовые колонки, которые точно хотим в числа ---
    numeric_cols = [
        "mp","starts","min","90s",
        "gls","ast","g_pk","pkatt",
        "crdy","crdr",
        "xg","npxg","xag","npxg_xag",
        "xg_per90","xag_per90","xg_xag_per90",
        "npxg_per90","npxg_xag_per90",
        "g_per90","a_per90","g_plus_a_per90",
        "g_plus_a_pk_per90","g_pk_per90"
    ]

    for col in numeric_cols:
        if col in out.columns:
            out[col] = out[col].apply(_to_number)

    # --- выкинуть строки без игрока или которые выглядят как агрегаты клуба ---
    # логика:
    # - если player NaN → дроп
    # - если player содержит "Squad Total" или "Matches" → дроп
    # - если мин нет (NaN или 0) → это часто строки про суммарный клуб или пустые строки
    def _is_agg_row(row):
        name = str(row.get("player", "")).strip().lower()
        if name in ["squad total", "matches", "match", "team total", "total"]:
            return True
        # иногда в таблицах fbref идёт повторяющийся заголовок с названием клуба в колонке player
        squad = str(row.get("squad", "")).strip().lower()
        if name == squad and row.get("mp") != row.get("mp"):  # mp is NaN
            return True
        return False

    mask_bad = out.apply(_is_agg_row, axis=1)

    # также отфильтруем строки у которых min пустой или 0
    if "min" in out.columns:
        mask_low_minutes = out["min"].isna() | (out["min"] == 0)
    else:
        mask_low_minutes = False

    before = len(out)
    out = out[~mask_bad & ~mask_low_minutes].copy()
    after = len(out)
    print(f"[CLEAN] убрано агрегатов/пустых строк: {before - after}")

    # --- удалить дубликаты по ключу (season, player, squad) ---
    if all(c in out.columns for c in ["season", "player", "squad"]):
        before = len(out)
        out = out.drop_duplicates(subset=["season","player","squad"])
        after = len(out)
        print(f"[DEDUP] удалено дублей: {before - after}")

    out.reset_index(drop=True, inplace=True)
    return out

In [4]:
# 3. Загрузка всех лиг и первичная сводка

all_leagues_raw = []
all_leagues_clean = []

for code in LEAGUES:
    print(f"\n=== {code.upper()} ===")
    df_raw = load_player_stats_for_league(code)
    print(f"[RAW] {code}: shape={df_raw.shape}")
    all_leagues_raw.append(df_raw)

    df_clean = clean_players_df(df_raw)
    print(f"[CLEANED] {code}: shape={df_clean.shape}")
    all_leagues_clean.append(df_clean)

players_all = pd.concat(all_leagues_clean, ignore_index=True)
print("\n=== MERGED PLAYERS_ALL ===")
print("shape:", players_all.shape)
print("cols:", list(players_all.columns))
players_all.head(3)


=== EPL ===
[LOAD] data/raw/fbref/epl_2020-2021/player_standard_stats.csv  shape=(553, 39)
[LOAD] data/raw/fbref/epl_2024-2025/player_standard_stats.csv  shape=(596, 39)
[LOAD] data/raw/fbref/epl_2023-2024/player_standard_stats.csv  shape=(603, 39)
[LOAD] data/raw/fbref/epl_2022-2023/player_standard_stats.csv  shape=(591, 39)
[LOAD] data/raw/fbref/epl_2021-2022/player_standard_stats.csv  shape=(567, 39)
[LOAD] data/raw/fbref/epl_2019-2020/player_standard_stats.csv  shape=(542, 39)
[RAW] epl: shape=(3452, 39)
[CLEAN] убрано агрегатов/пустых строк: 129
[DEDUP] удалено дублей: 0
[CLEANED] epl: shape=(3323, 39)

=== LALIGA ===
[LOAD] data/raw/fbref/laliga_2023-2024/player_standard_stats.csv  shape=(633, 39)
[LOAD] data/raw/fbref/laliga_2022-2023/player_standard_stats.csv  shape=(619, 39)
[LOAD] data/raw/fbref/laliga_2024-2025/player_standard_stats.csv  shape=(625, 39)
[LOAD] data/raw/fbref/laliga_2020-2021/player_standard_stats.csv  shape=(605, 39)
[LOAD] data/raw/fbref/laliga_2019-2020/p

,rk,player,nation,pos,squad,age,born,mp,starts,min,...,g_pk_per90,g_plus_a_pk_per90,xg_per90,xag_per90,xg_xag_per90,npxg_per90,npxg_xag_per90,matches,season,league
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,29.0,1990,22.0,20.0,1777.0,...,0.00,0.05,0.07,0.04,0.11,0.07,0.11,Matches,2020-2021,epl
1,2,Tammy Abraham,eng ENG,FW,Chelsea,22.0,1997,22.0,12.0,1040.0,...,0.52,0.61,0.51,0.06,0.57,0.51,0.57,Matches,2020-2021,epl
2,3,Che Adams,sct SCO,FW,Southampton,24.0,1996,36.0,30.0,2667.0,...,0.30,0.47,0.33,0.17,0.50,0.33,0.50,Matches,2020-2021,epl


In [5]:
# 4. Сохранение сводного датасета игроков

OUT_PLAYERS.parent.mkdir(parents=True, exist_ok=True)
players_all.to_csv(OUT_PLAYERS, index=False, encoding="utf-8")
print("✅ players_all сохранён в:", OUT_PLAYERS)

# Контроль: размер и пропуски
print("\nshape:", players_all.shape)
print("\nПропуски по top-колонкам:")
print(players_all[["player", "season", "league", "squad", "mp", "min", "gls", "ast", "xg", "xag"]].isna().sum())

✅ players_all сохранён в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/players/players_all.csv

shape: (9938, 39)

Пропуски по top-колонкам:
player    0
season    0
league    0
squad     0
mp        0
min       0
gls       0
ast       0
xg        0
xag       0
dtype: int64


In [6]:
# 5. Базовая проверка структуры и распределений

print("=== Игроки по лигам ===")
print(players_all["league"].value_counts())

print("\n=== Игроки по сезонам (топ-10) ===")
print(players_all["season"].value_counts().head(10))

print("\n=== Игроки по позициям (топ-10) ===")
print(players_all["pos"].value_counts().head(10))

print("\n=== Диапазон возраста ===")
print(players_all["age"].min(), "–", players_all["age"].max())

=== Игроки по лигам ===
league
laliga        3575
epl           3323
bundesliga    3040
Name: count, dtype: int64

=== Игроки по сезонам (топ-10) ===
season
2023-2024    1696
2021-2022    1686
2022-2023    1680
2024-2025    1667
2020-2021    1619
2019-2020    1590
Name: count, dtype: int64

=== Игроки по позициям (топ-10) ===
pos
DF       2933
MF       2043
FW       1346
FW,MF    1073
MF,FW     915
GK        726
DF,MF     390
MF,DF     268
DF,FW     158
FW,DF      86
Name: count, dtype: int64

=== Диапазон возраста ===
14.0 – 41.0


In [7]:
# 6. Контрольная сводка и сохранение метаданных

meta = {
    "rows_total": len(players_all),
    "cols_total": len(players_all.columns),
    "leagues": ", ".join(players_all["league"].unique()),
    "seasons": f"{players_all['season'].min()}–{players_all['season'].max()}",
    "age_range": f"{players_all['age'].min()}–{players_all['age'].max()}",
}

meta_path = OUT_PLAYERS.parent / "players_all_meta.txt"
with open(meta_path, "w", encoding="utf-8") as f:
    for k, v in meta.items():
        f.write(f"{k}: {v}\n")

print("✅ Метаданные сохранены в:", meta_path)
print("\n".join(f"{k}: {v}" for k, v in meta.items()))

✅ Метаданные сохранены в: /Users/kekc/Projects/GIT/sports-stats-analysis/data/processed/players/players_all_meta.txt
rows_total: 9938
cols_total: 39
leagues: epl, laliga, bundesliga
seasons: 2019-2020–2024-2025
age_range: 14.0–41.0
